**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import time
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

In [2]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [3]:
df_train = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')
df_test = pd.read_csv('../input/elo-merchant-category-recommendation/test.csv')
df_hist_trans = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')
df_merchants = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')

In [4]:
gdf = df_hist_trans.groupby("card_id")
print(type(gdf))
gdf = gdf.agg({'merchant_category_id':['min']}).reset_index()
print(type(gdf))
gdf.columns = ["card_id", "merchant_category_id"]
df_train = pd.merge(df_train, gdf, on="card_id", how="left")
df_test = pd.merge(df_test, gdf, on="card_id", how="left")

<class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
<class 'pandas.core.frame.DataFrame'>


In [5]:
gdf = df_hist_trans.groupby("card_id")
print(type(gdf))
gdf = gdf.agg({'merchant_category_id':['max']}).reset_index()
print(type(gdf))
gdf.columns = ["card_id", "max_merchant_category_id"]
df_train = pd.merge(df_train, gdf, on="card_id", how="left")
df_test = pd.merge(df_test, gdf, on="card_id", how="left")

<class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
<class 'pandas.core.frame.DataFrame'>


In [6]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [7]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [8]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()
del df_new_merchant_trans;gc.collect()

0

In [9]:
df_train.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'merchant_category_id', 'max_merchant_category_id',
       'new_hist_month_nunique', 'new_hist_hour_nunique',
       'new_hist_weekofyear_nunique', 'new_hist_dayofweek_nunique',
       'new_hist_year_nunique', 'new_hist_subsector_id_nunique',
       'new_hist_merchant_id_nunique', 'new_hist_merchant_category_id_nunique',
       'new_hist_purchase_amount_sum', 'new_hist_purchase_amount_max',
       'new_hist_purchase_amount_min', 'new_hist_purchase_amount_mean',
       'new_hist_purchase_amount_var', 'new_hist_installments_sum',
       'new_hist_installments_max', 'new_hist_installments_min',
       'new_hist_installments_mean', 'new_hist_installments_var',
       'new_hist_purchase_date_max', 'new_hist_purchase_date_min',
       'new_hist_month_lag_max', 'new_hist_month_lag_min',
       'new_hist_month_lag_mean', 'new_hist_month_lag_var',
       'new_hist_month_diff_mean', 'new_hist_weekend_

In [10]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

35

In [11]:
del df_hist_trans;gc.collect()
#del df_new_merchant_trans;gc.collect()#清理内存-
df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,merchant_category_id,max_merchant_category_id,new_hist_month_nunique,new_hist_hour_nunique,new_hist_weekofyear_nunique,new_hist_dayofweek_nunique,new_hist_year_nunique,new_hist_subsector_id_nunique,new_hist_merchant_id_nunique,new_hist_merchant_category_id_nunique,new_hist_purchase_amount_sum,new_hist_purchase_amount_max,new_hist_purchase_amount_min,new_hist_purchase_amount_mean,new_hist_purchase_amount_var,new_hist_installments_sum,new_hist_installments_max,new_hist_installments_min,new_hist_installments_mean,new_hist_installments_var,new_hist_purchase_date_max,new_hist_purchase_date_min,new_hist_month_lag_max,new_hist_month_lag_min,new_hist_month_lag_mean,new_hist_month_lag_var,new_hist_month_diff_mean,new_hist_weekend_sum,new_hist_weekend_mean,new_hist_category_1_sum,new_hist_category_1_mean,new_hist_card_id_size,new_hist_category_2_mean_mean,new_hist_category_3_mean_mean,new_hist_purchase_date_diff,new_hist_purchase_date_average,new_hist_purchase_date_uptonow,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_sum,hist_installments_max,hist_installments_min,hist_installments_mean,hist_installments_var,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_diff_mean,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,33,884,2.0,8.0,7.0,7.0,1.0,10.0,23.0,14.0,-13.244202,-0.296112,-0.724368,-0.575835,0.018445,0.0,0.0,0.0,0.000000,0.000000,2018-04-29 11:23:05,2018-03-05 14:04:36,2.0,1.0,1.478261,0.260870,16.695652,6.0,0.260870,0.0,0.000000,23.0,-0.550160,-0.592993,54.0,2.347826,443.0,9,23,35,7,2,21,94,41,-165.968739,2.258395,-0.739395,-0.638341,0.045003,4,1,0,0.015385,0.015206,2018-02-25 09:31:15,2017-06-27 14:18:08,0,-8,-3.911538,5.748901,16.773077,247,0.950000,90,0.346154,0,0.000000,260,0.072502,0.346719,242,0.930769,506
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,45,889,2.0,5.0,4.0,4.0,1.0,4.0,6.0,5.0,-4.355735,-0.701858,-0.739410,-0.725956,0.000205,6.0,1.0,1.0,1.000000,0.000000,2018-03-30 06:48:26,2018-02-01 17:07:54,2.0,1.0,1.500000,0.300000,17.833333,0.0,0.000000,0.0,0.000000,6.0,-0.550160,-0.606486,56.0,9.333333,473.0,12,24,50,7,2,24,142,57,-210.006336,4.630299,-0.742400,-0.600018,0.148200,543,10,-1,1.551429,2.282448,2018-01-31 22:31:09,2017-01-06 16:29:42,0,-12,-5.031429,14.477519,17.791429,339,0.968571,132,0.377143,31,0.088571,350,0.074568,-0.295163,390,1.114286,531
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,80,842,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.700326,-0.700326,-0.700326,-0.700326,NaN,0.0,0.0,0.0,0.000000,NaN,2018-04-28 17:43:11,2018-04-28 17:43:11,2.0,2.0,2.000000,NaN,16.000000,1.0,1.000000,0.0,0.000000,1.0,-0.549015,-0.592993,0.0,0.000000,444.0,10,14,22,7,2,7,13,8,-29.167391,-0.145847,-0.730138,-0.678311,0.007635,0,0,0,0.000000,0.000000,2018-02-27 19:08:25,2017-01-11 08:21:22,0,-13,-8.604651,14.768549,16.790698,41,0.953488,11,0.255814,0,0.000000,43,-0.087803,0.358458,412,9.581395,504
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,68,884,2.0,5.0,5.0,4.0,1.0,5.0,7.0,6.0,-4.654372,-0.566740,-0.734135,-0.664910,0.004340,5.0,1.0,-1.0,0.714286,0.571429,2018-04-18 11:00:11,2018-03-07 11:55:06,2.0,1.0,1.714286,0.238095,16.857143,3.0,0.428571,1.0,0.142857,7.0,-0.556518,-0.604559,41.0,5.857143,454.0,6,16,20,7,2,13,50,25,-49.491364,1.445596,-0.740897,-0.642745,0.068447,84,3,-1,1.090909,0.346890,2018-02-28 11:44:40,2017-09-26 16:22:21,0,-5,-2.831169,3.247437,

In [12]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
df_merchants['avg_sales_lag3']=imp.fit(pd.DataFrame(df_merchants['avg_sales_lag3'].values.reshape(-1,1))).transform(pd.DataFrame(df_merchants['avg_sales_lag3'].values.reshape(-1,1)))
df_merchants['avg_sales_lag6']=imp.fit(pd.DataFrame(df_merchants['avg_sales_lag6'].values.reshape(-1,1))).transform(pd.DataFrame(df_merchants['avg_sales_lag6'].values.reshape(-1,1)))
df_merchants['avg_sales_lag12']=imp.fit(pd.DataFrame(df_merchants['avg_sales_lag12'].values.reshape(-1,1))).transform(pd.DataFrame(df_merchants['avg_sales_lag12'].values.reshape(-1,1)))
df_merchants['category_2']=imp.fit(pd.DataFrame(df_merchants['category_2'].values.reshape(-1,1))).transform(pd.DataFrame(df_merchants['category_2'].values.reshape(-1,1)))
df_merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,1.00,1.666667,3,1.00,4.666667,6,1.00,3.833333,12,Y,-1,-1,1.0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,1.00,0.500000,3,1.00,0.361111,6,1.00,0.347222,12,Y,-1,-1,1.0


In [13]:
aggs={}
for col in ['avg_sales_lag3','avg_purchases_lag3','active_months_lag3','avg_sales_lag6','avg_purchases_lag6','active_months_lag6','avg_sales_lag12','avg_purchases_lag12','active_months_lag12','numerical_1','numerical_2']:
    aggs[col]= ['mean']
    
new_columns= get_new_columns('merchants',aggs)
df_merchants_group = df_merchants.groupby('merchant_category_id').agg(aggs)
df_merchants_group.columns = new_columns
df_merchants_group.reset_index(drop=False,inplace=True)
df_train=df_train.merge(df_merchants_group,on='merchant_category_id',how='left')
df_test=df_test.merge(df_merchants_group.reset_index(),on='merchant_category_id',how='left')
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,merchant_category_id,max_merchant_category_id,new_hist_month_nunique,new_hist_hour_nunique,new_hist_weekofyear_nunique,new_hist_dayofweek_nunique,new_hist_year_nunique,new_hist_subsector_id_nunique,new_hist_merchant_id_nunique,new_hist_merchant_category_id_nunique,new_hist_purchase_amount_sum,new_hist_purchase_amount_max,new_hist_purchase_amount_min,new_hist_purchase_amount_mean,new_hist_purchase_amount_var,new_hist_installments_sum,new_hist_installments_max,new_hist_installments_min,new_hist_installments_mean,new_hist_installments_var,new_hist_purchase_date_max,new_hist_purchase_date_min,new_hist_month_lag_max,new_hist_month_lag_min,new_hist_month_lag_mean,new_hist_month_lag_var,new_hist_month_diff_mean,new_hist_weekend_sum,new_hist_weekend_mean,new_hist_category_1_sum,new_hist_category_1_mean,new_hist_card_id_size,new_hist_category_2_mean_mean,new_hist_category_3_mean_mean,...,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_sum,hist_installments_max,hist_installments_min,hist_installments_mean,hist_installments_var,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_diff_mean,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,merchants_avg_sales_lag3_mean,merchants_avg_purchases_lag3_mean,merchants_active_months_lag3_mean,merchants_avg_sales_lag6_mean,merchants_avg_purchases_lag6_mean,merchants_active_months_lag6_mean,merchants_avg_sales_lag12_mean,merchants_avg_purchases_lag12_mean,merchants_active_months_lag12_mean,merchants_numerical_1_mean,merchants_numerical_2_mean
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,33,884,2.0,8.0,7.0,7.0,1.0,10.0,23.0,14.0,-13.244202,-0.296112,-0.724368,-0.575835,0.018445,0.0,0.0,0.0,0.000000,0.000000,2018-04-29 11:23:05,2018-03-05 14:04:36,2.0,1.0,1.478261,0.260870,16.695652,6.0,0.260870,0.0,0.000000,23.0,-0.550160,-0.592993,...,-165.968739,2.258395,-0.739395,-0.638341,0.045003,4,1,0,0.015385,0.015206,2018-02-25 09:31:15,2017-06-27 14:18:08,0,-8,-3.911538,5.748901,16.773077,247,0.950000,90,0.346154,0,0.000000,260,0.072502,0.346719,242,0.930769,506,1.459940,1.144017,3.000000,1.581098,1.230404,5.992840,1.680406,1.254399,11.898568,0.027540,0.026569
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,45,889,2.0,5.0,4.0,4.0,1.0,4.0,6.0,5.0,-4.355735,-0.701858,-0.739410,-0.725956,0.000205,6.0,1.0,1.0,1.000000,0.000000,2018-03-30 06:48:26,2018-02-01 17:07:54,2.0,1.0,1.500000,0.300000,17.833333,0.0,0.000000,0.0,0.000000,6.0,-0.550160,-0.606486,...,-210.006336,4.630299,-0.742400,-0.600018,0.148200,543,10,-1,1.551429,2.282448,2018-01-31 22:31:09,2017-01-06 16:29:42,0,-12,-5.031429,14.477519,17.791429,339,0.968571,132,0.377143,31,0.088571,350,0.074568,-0.295163,390,1.114286,531,1.414452,1.198175,2.998308,1.779147,1.339014,5.974475,2.005046,1.418842,11.785785,0.030126,0.028974
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,80,842,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.700326,-0.700326,-0.700326,-0.700326,NaN,0.0,0.0,0.0,0.000000,NaN,2018-04-28 17:43:11,2018-04-28 17:43:11,2.0,2.0,2.000000,NaN,16.000000,1.0,1.000000,0.0,0.000000,1.0,-0.549015,-0.592993,...,-29.167391,-0.145847,-0.730138,-0.678311,0.007635,0,0,0,0.000000,0.000000,2018-02-27 19:08:25,2017-01-11 08:21:22,0,-13,-8.604651,14.768549,16.790698,41,0.953488,11,0.255814,0,0.000000,43,-0.087803,0.358458,412,9.581395,504,2.198372,1.591046,2.995044,2.971746,1.997660,5.940459,3.437160,2.245974,11.535355,0.033576,0.030521
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,68,884,2.0,5.0,5.0,4.0,1.0,5.0,7.0,6.0,-4.654372,-0.566740,-0.734135,-0.664910,0.004340,5.0,1.0,-1.0,0.714286,0.571429,2018-04-18 11:00:11,2018-03-07 11:55:06

In [14]:
df_merchants['max_merchant_category_id']=df_merchants['merchant_category_id']
#df_test['max_merchant_category_id']=df_test['merchant_category_id']

aggs={}
for col in ['avg_sales_lag3','avg_purchases_lag3','active_months_lag3','avg_sales_lag6','avg_purchases_lag6','active_months_lag6','avg_sales_lag12','avg_purchases_lag12','active_months_lag12','numerical_1','numerical_2']:
    aggs[col]= ['mean']
    
new_columns= get_new_columns('max_merchants',aggs)
df_merchants_group = df_merchants.groupby('max_merchant_category_id').agg(aggs)
df_merchants_group.columns = new_columns
df_merchants_group.reset_index(drop=False,inplace=True)
df_train=df_train.merge(df_merchants_group,on='max_merchant_category_id',how='left')
df_test=df_test.merge(df_merchants_group.reset_index(),on='max_merchant_category_id',how='left')
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,merchant_category_id,max_merchant_category_id,new_hist_month_nunique,new_hist_hour_nunique,new_hist_weekofyear_nunique,new_hist_dayofweek_nunique,new_hist_year_nunique,new_hist_subsector_id_nunique,new_hist_merchant_id_nunique,new_hist_merchant_category_id_nunique,new_hist_purchase_amount_sum,new_hist_purchase_amount_max,new_hist_purchase_amount_min,new_hist_purchase_amount_mean,new_hist_purchase_amount_var,new_hist_installments_sum,new_hist_installments_max,new_hist_installments_min,new_hist_installments_mean,new_hist_installments_var,new_hist_purchase_date_max,new_hist_purchase_date_min,new_hist_month_lag_max,new_hist_month_lag_min,new_hist_month_lag_mean,new_hist_month_lag_var,new_hist_month_diff_mean,new_hist_weekend_sum,new_hist_weekend_mean,new_hist_category_1_sum,new_hist_category_1_mean,new_hist_card_id_size,new_hist_category_2_mean_mean,new_hist_category_3_mean_mean,...,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_diff_mean,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,merchants_avg_sales_lag3_mean,merchants_avg_purchases_lag3_mean,merchants_active_months_lag3_mean,merchants_avg_sales_lag6_mean,merchants_avg_purchases_lag6_mean,merchants_active_months_lag6_mean,merchants_avg_sales_lag12_mean,merchants_avg_purchases_lag12_mean,merchants_active_months_lag12_mean,merchants_numerical_1_mean,merchants_numerical_2_mean,max_merchants_avg_sales_lag3_mean,max_merchants_avg_purchases_lag3_mean,max_merchants_active_months_lag3_mean,max_merchants_avg_sales_lag6_mean,max_merchants_avg_purchases_lag6_mean,max_merchants_active_months_lag6_mean,max_merchants_avg_sales_lag12_mean,max_merchants_avg_purchases_lag12_mean,max_merchants_active_months_lag12_mean,max_merchants_numerical_1_mean,max_merchants_numerical_2_mean
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,33,884,2.0,8.0,7.0,7.0,1.0,10.0,23.0,14.0,-13.244202,-0.296112,-0.724368,-0.575835,0.018445,0.0,0.0,0.0,0.000000,0.000000,2018-04-29 11:23:05,2018-03-05 14:04:36,2.0,1.0,1.478261,0.260870,16.695652,6.0,0.260870,0.0,0.000000,23.0,-0.550160,-0.592993,...,2017-06-27 14:18:08,0,-8,-3.911538,5.748901,16.773077,247,0.950000,90,0.346154,0,0.000000,260,0.072502,0.346719,242,0.930769,506,1.459940,1.144017,3.000000,1.581098,1.230404,5.992840,1.680406,1.254399,11.898568,0.027540,0.026569,1.457113,1.240695,2.996249,1.803364,1.416167,5.949221,2.081724,1.611524,11.476822,0.016591,0.012986
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,45,889,2.0,5.0,4.0,4.0,1.0,4.0,6.0,5.0,-4.355735,-0.701858,-0.739410,-0.725956,0.000205,6.0,1.0,1.0,1.000000,0.000000,2018-03-30 06:48:26,2018-02-01 17:07:54,2.0,1.0,1.500000,0.300000,17.833333,0.0,0.000000,0.0,0.000000,6.0,-0.550160,-0.606486,...,2017-01-06 16:29:42,0,-12,-5.031429,14.477519,17.791429,339,0.968571,132,0.377143,31,0.088571,350,0.074568,-0.295163,390,1.114286,531,1.414452,1.198175,2.998308,1.779147,1.339014,5.974475,2.005046,1.418842,11.785785,0.030126,0.028974,1.091176,1.013048,3.000000,1.085686,1.035679,5.980392,1.219804,1.133544,11.823529,-0.053194,-0.053194
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,80,842,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.700326,-0.700326,-0.700326,-0.700326,NaN,0.0,0.0,0.0,0.000000,NaN,2018-04-28 17:43:11,2018-04-28 17:43:11,2.0,2.0,2.000000,NaN,16.000000,1.0,1.000000,0.0,0.000000,1.0,-0.549015,-0.592993,...,2017-01-11 08:21:22,0,-13,-8.604651,14.768549,16.790698,41,0.953488,11,0.255814,0,0.000000,43,-0.087803,0.358458,412,9.581395,504,2.198372,1.591046,2.995044,2.971746,1.997660,5.940459,3.437160,2.245974,11.535355,0.033576,0.030521,3.212476,1.723277,2.992606,3.934726,2.134505,5.921621,4.508767,2.393938,11.408163,0.067936,0.067353
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,68,884,

In [15]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [16]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)


In [17]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']
del df_train['target']

In [18]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 27, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.015,
         "min_child_samples": 50,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

fold 0
Training until validation scores don't improve for 200 rounds.
[200]	training's rmse: 3.53049	valid_1's rmse: 3.69205
[400]	training's rmse: 3.43644	valid_1's rmse: 3.67773
[600]	training's rmse: 3.37174	valid_1's rmse: 3.67413
[800]	training's rmse: 3.32032	valid_1's rmse: 3.6725
[1000]	training's rmse: 3.27357	valid_1's rmse: 3.67347
Early stopping, best iteration is:
[836]	training's rmse: 3.31119	valid_1's rmse: 3.67217
fold 1
Training until validation scores don't improve for 200 rounds.
[200]	training's rmse: 3.54547	valid_1's rmse: 3.65432
[400]	training's rmse: 3.45052	valid_1's rmse: 3.63618
[600]	training's rmse: 3.38689	valid_1's rmse: 3.63219
[800]	training's rmse: 3.3368	valid_1's rmse: 3.63133
[1000]	training's rmse: 3.28916	valid_1's rmse: 3.63075
[1200]	training's rmse: 3.24699	valid_1's rmse: 3.6302
[1400]	training's rmse: 3.20719	valid_1's rmse: 3.63002
[1600]	training's rmse: 3.16865	valid_1's rmse: 3.63009
Early stopping, best iteration is:
[1449]	training's 

3.6567212247367635

In [19]:
model_without_outliers = pd.DataFrame({"card_id":df_test["card_id"].values})
model_without_outliers["target"] = predictions

In [20]:
#del df_train['outliers']
#del df_train['target']
#target = df_train['outliers']

In [21]:
features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month']]
categorical_feats = [c for c in features if 'feature_' in c]

In [22]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333}

In [23]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
%%time
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

start = time.time()


for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(log_loss(target, oof)))

UsageError: Line magic function `%%time` not found.


In [24]:
### 'target' is the probability of whether an observation is an outlier
df_outlier_prob = pd.DataFrame({"card_id":df_test["card_id"].values})
df_outlier_prob["target"] = predictions
df_outlier_prob.head()

,card_id,target
0,C_ID_0ab67a22ab,-2.859362
1,C_ID_130fd0cbdd,-0.300522
2,C_ID_b709037bc5,-0.941609
3,C_ID_d27d835a9f,-0.169592
4,C_ID_2b5e3df5c2,-1.281567


In [25]:
outlier_id = pd.DataFrame(df_outlier_prob.sort_values(by='target',ascending = False).head(25000)['card_id'])

In [26]:
best_submission = pd.read_csv('../input/finaldata/submission_ashish.csv')
most_likely_liers = best_submission.merge(outlier_id,how='right')
most_likely_liers.head()

,card_id,target
0,C_ID_f7cada36d3,0.440054
1,C_ID_f72513589e,0.668335
2,C_ID_a7f2d6984e,0.332898
3,C_ID_83e740197f,1.166411
4,C_ID_5005a6ad0e,0.676176


In [27]:
%%time
for card_id in most_likely_liers['card_id']:
    model_without_outliers.loc[model_without_outliers['card_id']==card_id,'target']\
    = most_likely_liers.loc[most_likely_liers['card_id']==card_id,'target'].values

CPU times: user 6min 9s, sys: 236 ms, total: 6min 9s
Wall time: 6min 9s


In [28]:
model_without_outliers.to_csv("combining_submission.csv", index=False)

In [29]:
#sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
#sub_df["target"] = predictions
#sub_df.to_csv("submission.csv", index=False)

In [30]:
#cols = (feature_importance_df[["Feature", "importance"]]
        #.groupby("Feature")
        #.mean()
        #.sort_values(by="importance", ascending=False)[:1000].index)

#best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

#plt.figure(figsize=(14,25))
#sns.barplot(x="importance",
            #y="Feature",
            #data=best_features.sort_values(by="importance",
                                          # ascending=False))
#plt.title('LightGBM Features (avg over folds)')
#plt.tight_layout()
#plt.savefig('lgbm_importances.png')

**To be continued ...**